### 1. Load the Data

In [83]:
import pandas as pd
import os

# Define the base path for the data folder
data_folder = os.path.join(os.path.dirname(os.getcwd()), 'data')

# Load the CSV files
content_df = pd.read_csv(os.path.join(data_folder, 'Content.csv'))
reaction_df = pd.read_csv(os.path.join(data_folder, 'Reactions.csv'))
reaction_types_df = pd.read_csv(os.path.join(data_folder, 'ReactionTypes.csv'))
reaction_df.rename(columns={'Type': 'Content Type'}, inplace=True)
reaction_types_df.rename(columns={'Type': 'Reaction Type'}, inplace=True)
reaction_types_df.rename(columns={'Reaction Type': 'Content Type'}, inplace=True)

### 2. Remove Rows with Missing Values

In [84]:
import numpy as np

# Clean missing values in Reaction dataset
reaction_df.replace(['', ' ', 'null', 'N/A', '-'], np.nan, inplace=True)

print("Missing values after standardization:Reaction")
print(reaction_df.isnull().sum())

reaction_df_cleaned = reaction_df.dropna()

print("Missing values after cleaning:Reaction")
print(reaction_df_cleaned.isnull().sum())

# Clean missing values in Content dataset
content_df.replace(['', ' ', 'null', 'N/A', '-'], np.nan, inplace=True)

print("Missing values after standardization (Content):")
print(content_df.isnull().sum())

content_df_cleaned = content_df.dropna()

print("Missing values after cleaning (Content):")
print(content_df_cleaned.isnull().sum())

# Clean missing values in ReactionTypes dataset
reaction_types_df.replace(['', ' ', 'null', 'N/A', '-'], np.nan, inplace=True)

print("Missing values after standardization (ReactionTypes):")
print(reaction_types_df.isnull().sum())

reaction_types_df_cleaned = reaction_types_df.dropna()

print("Missing values after cleaning (ReactionTypes):")
print(reaction_types_df_cleaned.isnull().sum())

Missing values after standardization:Reaction
Content ID        0
Content Type    980
Datetime          0
dtype: int64
Missing values after cleaning:Reaction
Content ID      0
Content Type    0
Datetime        0
dtype: int64
Missing values after standardization (Content):
Content ID    0
Category      0
dtype: int64
Missing values after cleaning (Content):
Content ID    0
Category      0
dtype: int64
Missing values after standardization (ReactionTypes):
Content Type    0
Score           0
dtype: int64
Missing values after cleaning (ReactionTypes):
Content Type    0
Score           0
dtype: int64


### 3. Change Data Types

In [85]:
# Convert 'Datetime' to datetime format
reaction_df['Datetime'] = pd.to_datetime(reaction_df['Datetime'])

# Ensure 'Score' is numeric
reaction_types_df['Score'] = pd.to_numeric(reaction_types_df['Score'], errors='coerce')

### 4. Remove Irrelevant Columns

In [87]:
# Keep relevant columns
content_df = content_df[['Content ID', 'Category']]
reaction_df = reaction_df[['Content ID', 'Content Type', 'Datetime']]
reaction_types_df = reaction_types_df[['Content Type', 'Score']]

### 5. Remove quotes marks from categories

In [88]:
# Remove both single and double quotation marks from the 'Category' column
content_df['Category'] = content_df['Category'].str.replace('"', '', regex=False)  # Remove double quotes
content_df['Category'] = content_df['Category'].str.replace("'", '', regex=False)  # Remove single quotes

In [89]:
print(content_df.columns)
print(reaction_df.columns)
print(reaction_types_df.columns)
print(content_df['Category'].head())

Index(['Content ID', 'Category'], dtype='object')
Index(['Content ID', 'Content Type', 'Datetime'], dtype='object')
Index(['Content Type', 'Score'], dtype='object')
0          Studying
1    healthy eating
2    healthy eating
3        technology
4              food
Name: Category, dtype: object


### 6. Save the updated files

In [90]:
# Save the updated content_df to the CSV file in the 'data' folder
content_df.to_csv(os.path.join(data_folder, 'Content.csv'), index=False)

# Save the updated reaction_df to the same CSV file, replacing the existing one
reaction_df.to_csv(os.path.join(data_folder, 'Reactions.csv'), index=False)

# Save the updated reaction_types_df to the same CSV file, replacing the existing one
reaction_types_df.to_csv(os.path.join(data_folder, 'ReactionTypes.csv'), index=False)

### 7. Merging the Datasets

In [91]:
# Merge Reaction with Content
merged_df = reaction_df.merge(content_df, on='Content ID', how='inner')

# Merge the result with Reaction Types
final_df = merged_df.merge(reaction_types_df, on='Content Type', how='inner')

In [94]:
final_df.rename(columns={'Content Type': 'Reaction Type'}, inplace=True)
print(final_df.tail())

                                 Content ID Reaction Type            Datetime  \
24568  435007a5-6261-4d8b-b0a4-55fdc189754b         adore 2020-10-04 22:26:33   
24569  435007a5-6261-4d8b-b0a4-55fdc189754b         adore 2020-09-18 10:50:50   
24570  4e4c9690-c013-4ee7-9e66-943d8cbd27b7         adore 2020-10-31 03:58:44   
24571  4e4c9690-c013-4ee7-9e66-943d8cbd27b7         adore 2020-06-25 15:12:29   
24572  4e4c9690-c013-4ee7-9e66-943d8cbd27b7         adore 2020-12-17 16:32:57   

       Category  Score  
24568  veganism     72  
24569  veganism     72  
24570   culture     72  
24571   culture     72  
24572   culture     72  


### 8. Calculate Total Scores for Each Category

In [95]:
# Group by Category and sum the scores
category_scores = final_df.groupby('Category')['Score'].sum().reset_index()

In [96]:
# Rename the column for clarity
category_scores.rename(columns={'Score': 'Total Score'}, inplace=True)

### 9. Extract the Top 5 Categories

In [97]:
# Sort by Total Score and get the top 5
top_categories = category_scores.sort_values(by='Total Score', ascending=False).head(5)

In [98]:
print(top_categories)

          Category  Total Score
13         animals        71242
20  healthy eating        69067
25      technology        68521
22         science        66549
15         culture        64952


### 10. Save the Final Dataset and Top Categories

In [99]:
# Save the final merged dataset
final_df.to_csv(os.path.join(data_folder, 'Final_Dataset.csv'), index=False)

# Save the top 5 categories
top_categories.to_csv(os.path.join(data_folder, 'Top_5_Categories.csv'), index=False)